# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [75]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import udf
from pyspark.sql.functions import concat
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StringType
from pyspark.ml.feature import CountVectorizer,IDF,VectorAssembler,StandardScaler\
                               ,MinMaxScaler
# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [76]:
from pyspark.ml.feature import Normalizer

In [77]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [78]:
stack_overflow_data = 'Train_onetag_small.json'

In [79]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [80]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [81]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [82]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [83]:
# TODO: write your code to answer question 1
df.select('BodyLength').filter(df.Id==1112).show()

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [84]:
# TODO: write your code to answer question 2
df = df.withColumn('TitleBody',concat(df.Title,df.Body))

In [85]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [86]:
regexTokenizer2 = RegexTokenizer(inputCol="TitleBody", outputCol="TitleBodyTokens", pattern="\\W")
df = regexTokenizer2.transform(df)

In [87]:
TitleBodyLen=udf(lambda x: len(x) ,IntegerType())

In [88]:
df = df.withColumn('TitleBodyLen',TitleBodyLen(df.TitleBodyTokens))

In [89]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [90]:
df.select('TitleBodyLen').filter(df.Id==5123).show()

+------------+
|TitleBodyLen|
+------------+
|         135|
+------------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [91]:
# TODO: write your code to create this vector

assembler = VectorAssembler(inputCols=["TitleBodyLen"], outputCol="NumFeatures")
df = assembler.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [92]:
# TODO: write your code to answer question 3
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures2")
df = scaler.transform(df)

In [93]:
df.select('ScaledNumFeatures2').filter(df.Id==512).show()

+------------------+
|ScaledNumFeatures2|
+------------------+
|             [1.0]|
+------------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [94]:

scaler2 = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures3",withStd=False,withMean=False)
scalerModel = scaler2.fit(df)
df = scalerModel.transform(df)# TODO: write your code to answer question 4

In [95]:
df.select('ScaledNumFeatures3').filter(df.Id==512).show()

+------------------+
|ScaledNumFeatures3|
+------------------+
|            [57.0]|
+------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [ ]:
# TODO: write your code to answer question 5
scaler3 = MinMaxScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures4")
scalerModel = scaler3.fit(df)
df = scalerModel.transform(df)# TODO: write your code to answer question 4

In [ ]:
df.select('ScaledNumFeatures4').filter(df.Id==512).show()